In [1]:
import cv2
import torch
import torch.nn.functional as F
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image
import numpy as np
import time

c:\Users\Dell\miniconda3\envs\facerec\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def make_entry (vectorDB,mtcnn,resnet,device): 
    id_ = input("Enter your ID")

    a = {id_: {
                "id_vectors": []
                }
            }
    
    print("Recording...") 
    a[id_]["id_vectors"] =  get_vector(mtcnn,resnet,device)
    print(f"Samples---{len(a[id_]["id_vectors"] )} running on {device}")
    vectorDB.update(a)


def get_vector(mtcnn,resnet,device):
    vectors = []
    cap = cv2.VideoCapture(0)

   
    n_samples = 50

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break


        
        percentage = (len(vectors)/n_samples)*100
        
        
        cv2.putText(frame, f"Recording: {int(percentage)}%", (10, frame.shape[0] - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame_rgb)

    
        face_tensor = mtcnn(img)

        
        boxes, _ = mtcnn.detect(img)
        if boxes is not None:
            for box in boxes:
                box = [int(b) for b in box]
                cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

        
        if face_tensor is not None:
            face_tensor = face_tensor.unsqueeze(0).to(device)
            with torch.no_grad():
                embedding = resnet(face_tensor)
            
            embedding_np = embedding.cpu().numpy().flatten()
            cv2.putText(frame, f"Recording data...", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            vectors.append(embedding_np)
        else:
            cv2.putText(frame, "No face detected", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        
        cv2.imshow('Live Face Authentication', frame)
        
        
        if percentage >= 98 or cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return torch.tensor(np.array(vectors))
        

        
        

In [6]:
vectorDB = dict()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(image_size=160, margin=20, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [9]:
make_entry (vectorDB,mtcnn,resnet,device)

Recording...
Samples---50 running on cuda


In [10]:
vectorDB.keys()

dict_keys(['1', '2', '3'])

In [15]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(frame_rgb)
    face_tensor = mtcnn(img)
    if face_tensor is not None:
        face_tensor = face_tensor.unsqueeze(0).to(device)
        with torch.no_grad():
            embedding = resnet(face_tensor)
        
        embedding_np = embedding.cpu().numpy().flatten()
    if  cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

Failed to grab frame


In [11]:
def cal_cosine(source_vec, target_vec):
    cosine_sim = F.cosine_similarity(source_vec, target_vec, dim=1)
    return torch.mean(cosine_sim)

In [22]:
cal_cosine(source_vec, target_vec).item()

0.8181321024894714

In [31]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(frame_rgb)
    face_tensor = mtcnn(img)
    if face_tensor is not None:
        face_tensor = face_tensor.unsqueeze(0).to(device)
        with torch.no_grad():
            embedding = resnet(face_tensor)
        
        embedding_np = embedding.cpu().detach().flatten()
        id_,val = search_DB(embedding_np)
        print(id_,val)

    
    cv2.imshow("Webcam Feed", frame)

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



1 0.07336177676916122
1 0.018844662234187126
1 0.05815839767456055
1 0.0872763991355896
1 0.09036781638860703
1 0.11031754314899445
1 0.10677248239517212
1 0.03559480234980583
1 0.09543789178133011
1 0.11109425872564316
1 0.10678990185260773
1 0.13766157627105713
3 0.588904619216919
3 0.5905807614326477
1 0.31200283765792847
1 0.40272057056427
1 0.12003380060195923
1 0.13414879143238068
1 0.18388324975967407
1 0.20881590247154236
1 0.17436911165714264
1 0.15250298380851746
1 0.1960173398256302
1 0.3311907947063446
1 0.23497070372104645
1 0.2868334949016571
1 0.13887324929237366
1 0.09863264113664627
1 0.1416199505329132
1 0.10928623378276825
1 0.08442482352256775
1 0.1071598157286644
1 0.09740563482046127
1 0.12177770584821701
1 0.07537279278039932
1 0.14314430952072144
1 0.1415897011756897
1 0.08397383987903595
1 0.12647698819637299
1 0.11023363471031189
1 0.09782392531633377
1 0.36540713906288147
1 0.46421128511428833
1 0.14764265716075897
1 0.25197529792785645
1 0.22120679914951324


In [28]:
def search_DB(ref = None):
    matched = dict()
    for k in vectorDB.keys():
        target = vectorDB[k]["id_vectors"]
        sim = cal_cosine(ref, target).item()
        matched.update({k:sim})
    id_ = max(matched, key=matched.get)
    val = matched[k]

    return id_,val
        
